In [ ]:
- hosts: all
  tasks:
  - name: ping all hosts
    ping:

  - name: Docker install
    become: yes
    apt:
      name: docker.io
      state: latest

  - name: Java install
    become: yes
    apt:
      name: default-jre
      state: latest
    tags:
      - java_install

  - name: Supervisor install
    become: yes
    apt:
      name: supervisor
      state: latest
    tags:
      - supervisor_install

  - name: Download Anaconda python3
    get_url:
      url: https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
      dest: ~/anaconda_installer.sh

  - name: Download Flow
    get_url:
      url: https://h2o-release.s3.amazonaws.com/h2o/rel-yu/2/h2o-3.28.0.2.zip
      dest: ~/h2o-3.28.0.2.zip

  - name: create folder for H2o
    file:
       path: ~/H2O
       state: directory

  - name: Unzip Flow
    unarchive:
      src: ~/h2o-3.28.0.2.zip
      dest: ~/H2O
      remote_src: yes

  - name: Chmod anaconda
    command: chmod +x ~/anaconda_installer.sh

  - stat:
      path: ~/Anaconda
    register: result

  - name: Install anaconda
    command: bash ~/anaconda_installer.sh -b -p ~/Anaconda
    when: result.stat.exists == false

  - name: anaconda_path
    lineinfile:
      path: ~/.bashrc
      regexp: '^export PATH='
      line: export PATH=~/Anaconda/bin:$PATH

  - stat:
      path: ~/Anaconda/envs/keras_env
    register: env_result

  - name: create_env
    command: conda create -c h2oai -y -n keras_env h2o tensorflow-gpu=1.12.0 tensorflow-hub keras=2.2.4 pandas=0.24.1 pyyaml=3.13 scikit-learn=0.20.2 matplotlib jupyter jupyterlab
    environment:
      PATH: '{{ ansible_env.HOME }}/Anaconda/bin:{{ ansible_env.PATH }}'
    when: env_result.stat.exists == false

  - name: start supervisor service
    become: yes
    service:
      name: supervisor
      state: started

  - name: copy_supervisor_conf
    become: yes
    copy:
      src: jupyter.conf
      dest: /etc/supervisor/conf.d/jupyter.conf
    tags:
        - conf_filecopy

  - name: copy_supervisor_flow_conf
    become: yes
    copy:
      src: flow.conf
      dest: /etc/supervisor/conf.d/flow.conf
    tags:
        - conf_filecopy

  - name: Create a directory if it does not exist
    file:
      path: /home/ubuntu/keras
      state: directory
      mode: '0755'

  - name: supervisor reload
    become: yes
    command: supervisorctl reload
    tags:
      - supervisor_reload

  - name: start jupyter
    become: yes
    supervisorctl:
      name: jupyter:jupyter-0
      state: started
    tags:
        - jupytertf2_start

  - name: start flow
    become: yes
    supervisorctl:
      name: flow:flow-0
      state: stopped
    tags:
        - flow_start

  - name: copy_python_code
    copy:
      src: ../Intro.ipynb
      dest: ~/keras/Intro.ipynb
    tags:
        - filecopy